In [1]:
import geemap
import ee
ee.Initialize()
reducer = ee.Reducer.mean()
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
'''
U1  :2m处观测的风速（m/s）(栅格数据)   影像集
air  :空气密度（kg/m3)(栅格数据)    影像集
SR   :太阳辐射总量（cal/cm2)     影像集
tem  :平均气温（摄氏度）         影像集
R    :降水量（mm）            影像集 
I=0
Rd=10
Nd=8
dem    :高程（m）      影像
Sand   :土壤砂粒含量（%）影像集
Silt    :土壤粉粒含量（%）影像集
clay    :黏土含量（%）     影像集
OM      :有机质含量（%）    影像集
K       :土壤粗糙度因子（无量纲）
NDVI    :归一化植被指数     影像集

'''

In [ ]:
'''
定义函数区
'''
def mask(image):                   #替换空值
    return image.unmask(-999)
def upmask(image):           #将<0的部分都改为0
    return image.updateMask(image.gte(0))
    
def NDVI_V3(img):           #计算ndvi的值，这里是lansat8卫星的计算方式
 ndvi = img.normalizedDifference(["B5","B4"])
 return ndvi



In [ ]:
def mode(U1,air,SR,tem,R,dem,sand,silt,clay,om,K,NDVI,I=0,Rd=10,Nd=8):
    
    WE = image.expression(
    '(U1*(U1-5)*(U1-5)*8)*(air/9.32)', {
        'U1': U1,
        'air': air,
    })
    
    ETp=image.expression(
    '0.0162*(SR*58.5)*(tem+17.8)', {
        'SR': SR,
        'tem': tem,
    })
    
    SW=image.expression(
    '(ETp-(R+I)*(Rd/Nd))/ETp', {
        'ETp': ETp,
        'R': R,
        'I':I;
        'Rd':Rd,
        'Nd':Nd,
    })
    
    p=image.expression(
    '101.3*((293-0.0065*dem)/293)**5.26', {
        'dem': dem,
    })
    
    SD=image.expression(
    '1-p', {
        'p': p,
    })
    
    WF=WE*SW*SD
    
    som=image.expression(
    'om * (om >= 0) / 100', {
        'om': om,
    })
    
    EF=image.expression(
    '(29.09+0.31*sand+0.17*silt+(0.33*(sand/clay))-2.59*som-0.95*0)/100', {
        'sand': sand,
        'silt':silt,
        'clay':clay,
        'som':som,
    })
    
    mask(EF)
    
    upmask(EF)
    
    SCF=image.expression(
    '1/(1+0.0066*((clay)**2)+0.21*((som)**2))', {
        'clay':clay,
        'som':som,
    })
    
    Fc=image.expression(
    '2.71828**((-2*NDVI)/(1-NDVI))', {
        'NDVI': NDVI,
    })
    
    COG=image.expression(
    '2.71828**(-5.614*(Fc^0.7366))', {
        'Fc': Fc,
    })
    
    Qmax=image.expression(
    '109.8*(WF*EF*SCF*K*COG)', {
        'WF': WF,
        'EF': EF,
        'SCF':SCF,
        'K': K,
        'COG':COG,
    
    S=image.expression(
    '150.71*((WF*EF*SCF*K*COG)**0.3711)', {
        'WF': WF,
        'EF': EF,
        'SCF':SCF,
        'K': K,
        'COG':COG,
        
    Qx=image.expression(
    'Qmax*(1-2.71828**(((100/S)**2)))', {
        'Qmax': Qmax,
        'S':S,

    })
        
        
    SL=Qx/100
        
    return SL

In [11]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [ ]:
def main(start_year,start_month=1,start_day=1,end_year,end_month=12,end_day=31,):
    start_time=  str(start_year) + '-' + str("%02d" % start_month) + '-' + str("%02d" % start_day)
    end_time=  str(start_year) + '-' + str("%02d" % end_month) + '-' + str("%02d" % end_day)
    